# Building Autonomous Trader using mt5b3

### Installing mt5b3
### 1. Download it from https://github.com/paulo-al-castro/mt5b3/
### 2. unzip it in a proper fold
### 3. Install Metatrader 5 (https://www.metatrader5.com/)
### 4. Install python package Metatrader5 using pip
####   Use: pip install MetaTrader5 ... or Use sys package




In [ ]:
# installing Metatrader5 using sys
import sys
# python MetaTrader5
!{sys.executable} -m pip install Metatrader5
#mt5b3 
!{sys.executable} -m pip install mt5b3

In [1]:
import mt5b3 as b3
import pandas as pd
connected=b3.connect()
if connected:
    print('Ok!! It is connected to B3 exchange!!')
else:
    print('Something went wrong! It is NOT connected to B3!!')



Ok!! It is connected to B3 exchange!!


## Let's create a Simple Trader based on AI (decision tree)

In [2]:
## Defines the Simple AI Trader
from sklearn import tree
from sklearn.preprocessing import KBinsDiscretizer

class SimpleAITrader(b3.Trader):

    def setup(self,dbars):
        assets=list(dbars.keys())
        if len(assets)!=1:
            print('Error, this trader is supposed to deal with just one asset')
            return None
        bars=dbars[assets[0]]
        # remove irrelevant info
        if 'time' in bars:
            del bars['time']
        timeFrame=10 # it takes into account the last 10 bars
        horizon=1 # it project the closing price for next bar
        target='close' # name of the target column
        ds=b3.ai_utils.bars2Dataset(bars,target,timeFrame,horizon)
        
        X=b3.ai_utils.fromDs2NpArrayAllBut(ds,['target'])
        discretizer = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='uniform') 
        # creates the discrete target
        ds['target']=b3.ai_utils.discTarget(discretizer,ds['target'])
        Y=b3.ai_utils.fromDs2NpArray(ds,['target'])

        clf = tree.DecisionTreeClassifier()

        clf = clf.fit(X, Y)
        self.clf=clf

    def trade(self,bts,dbars):
            assets=dbars.keys()
            orders=[]
            timeFrame=10 # it takes into account the last 10 bars
            horizon=1 # it project the closing price for next bar
            target='close' # name of the target column
            for asset in assets:
                curr_shares=b3.backtest.getShares(bts,asset)
                money=b3.backtest.getBalance(bts)/len(assets) # divide o saldo em dinheiro igualmente entre os ativos
                free_shares=b3.backtest.getAfforShares(bts,dbars,asset)
                # get new information (bars), transform it in X
                bars=dbars[asset]
                #remove irrelevant info
                if 'time' in bars:
                    del bars['time']
                # convert from bars to dataset
                ds=b3.ai_utils.bars2Dataset(bars,target,timeFrame,horizon)
                # Get X fields
                X=b3.ai_utils.fromDs2NpArrayAllBut(ds,['target'])

                # predict the result, using the latest info
                p=self.clf.predict([X[-1]])
                if p==2:
                    #buy it
                    order=b3.buyOrder(asset,free_shares)
                elif p==0:
                    #sell it
                    order=b3.sellOrder(asset,curr_shares)
                else:
                    order=None
                if order!=None:
                    orders.append(order)
            return orders    

# creates instance of the Simple AI Trading
trader=SimpleAITrader()
print(trader)


## Setup and Run a backtest!

In [3]:
# sets Backtest options 
prestart=b3.date(2018,12,10)
start=b3.date(2019,1,10)
end=b3.date(2019,2,27)
capital=100000
results_file='data_equity_file.csv'
verbose=False 
assets=['PETR4']
# Use True if you want debug information for your Trader 
#sets the backtest setup
period=b3.DAILY 
 # it may be b3.INTRADAY (one minute interval)
bts=b3.backtest.set(assets,prestart,start,end,period,capital,results_file,verbose)
if b3.backtest.checkBTS(bts): # check if the backtest setup is ok!
    print('Backtest Setup is Ok!')
else:
    print('Backtest Setup is NOT Ok!')



Backtest Setup is Ok!


In [4]:
# Running the backtest
df= b3.backtest.run(trader,bts)   
# run calls the Trader. setup and trade (one for bar)

# evaluates the backtest results
b3.backtest.evaluate(df)


End of backtest with  33  bars,  saving equity file in  data_equity_file.csv

 -----------------------   Backtest Report  ------------------------------- 

Total Return (%)=8.34 in 33 bars 
Average Bar Return (%)=0.25  
Std Deviation of returns (%) =1.5111

 ----------------------        End of Report     -------------------------------- 



## Evaluating Backtesting results

The method backtest.run creates a data file with the name given in the backtest setup (bts) 

This will give you a report about the trader performance

We need ot note that it is hard to perform meaningful evaluations using backtest. There are many pitfalls to avoid and it may be easier to get trading robots with great performance in backtest, but that perform really badly in real operations. 

More about that in mt5b3 backtest evaluation chapter.

For a deeper discussion, we suggest:
Is it a great Autonomous Trading Strategy or you are just fooling yourself Bernardini,M. and Castro, P.A.L

In order to analyze the trader's backtest, you may use :

b3.backtest.evaluateFile(fileName)  
#fileName is the name of file generated by the backtest

or

b3.bactest.evaluate(df)  # df is the dataframe returned by b3.backtest.run
